In [144]:
import pandas as pd
import plotly.express as px
from pandas_datareader import data as pdr
import yfinance as yf

In [213]:
def extract_airline_data(airline_name,name):
    dict_airline = {}
    for i in range(2010, 2023):
        dict_airline[i] = {}
        file_name = f'dados/resumo_anual_{i}.csv'
        df = pd.read_csv(file_name, sep=';', encoding='ISO-8859-1', decimal = ',')
        df = df[['EMPRESA (SIGLA)', 'MÊS', 'NATUREZA', 'PASSAGEIROS PAGOS', 'ASSENTOS', 'RPK', 'ASK', 'DECOLAGENS', 'DISTÂNCIA VOADA (KM)', 'COMBUSTÍVEL (LITROS)']]
        df_airline = df[df['EMPRESA (SIGLA)'] == airline_name]
        df_airline = df_airline.dropna()

        # Assentos Totais
        assentos_totais = int(df_airline['ASSENTOS'].sum())
        dict_airline[i]['assentos_totais'] = assentos_totais
        
        #RPK Total
        rpk_total = df_airline['RPK']
        rpk_total = rpk_total.astype(float)
        rpk_total = rpk_total.sum()
        dict_airline[i]['rpk_total'] = rpk_total
        
        #ASK Total
        ask_total = df_airline['ASK']
        ask_total = ask_total.astype(float)
        ask_total = ask_total.sum()
        dict_airline[i]['ask_total'] = ask_total

        # Passageiros Pagos
        passageiros_pgs = int(df_airline['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs'] = passageiros_pgs

        # Passageiros Totais Doméstica
        domestica = df_airline[df_airline['NATUREZA'] == 'DOMÉSTICA']
        assentos_domestica = int(domestica['ASSENTOS'].sum())
        dict_airline[i]['assentos_domestica'] = assentos_domestica

        # Passageiros Pagos Doméstica
        passageiros_pgs_domestica = int(domestica['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs_domestica'] = passageiros_pgs_domestica

        # Assentos doméstica desocupados
        assentos_domestica_desocupados = assentos_domestica - passageiros_pgs_domestica

        # Vacância doméstica
        if assentos_domestica != 0:
            vacancia_domestica = (assentos_domestica_desocupados / assentos_domestica) * 100
        else:
            vacancia_domestica = 0
        dict_airline[i]['vacancia_domestica'] = vacancia_domestica
        
        #RPK Doméstica
        rpk_domestica = domestica['RPK']
        rpk_domestica = rpk_domestica.astype(float)
        rpk_domestica = rpk_domestica.sum()
        dict_airline[i]['rpk_domestica'] = rpk_domestica
        
        #ASK Doméstica
        ask_domestica = domestica['ASK']
        ask_domestica = ask_domestica.astype(float)
        ask_domestica = ask_domestica.sum()
        dict_airline[i]['ask_domestica'] = ask_domestica

        # Passageiros Totais Internacional
        internacional = df_airline[df_airline['NATUREZA'] == 'INTERNACIONAL']
        assentos_internacional = int(internacional['ASSENTOS'].sum())
        dict_airline[i]['assentos_internacional'] = assentos_internacional

        # Passageiros Pagos Internacional
        passageiros_pgs_internacional = int(internacional['PASSAGEIROS PAGOS'].sum())
        dict_airline[i]['passageiros_pgs_internacional'] = passageiros_pgs_internacional

        # Assentos desocupados Internacional
        assentos_internacional_desocupados = assentos_internacional - passageiros_pgs_internacional

        # Vacância Internacional
        if assentos_internacional != 0:
            vacancia_internacional = (assentos_internacional_desocupados / assentos_internacional) * 100
        else:
            vacancia_internacional = 0
        dict_airline[i]['vacancia_internacional'] = vacancia_internacional
        
        #RPK Internacional
        rpk_internacional = internacional['RPK']
        rpk_internacional = rpk_internacional.astype(float)
        rpk_internacional = rpk_internacional.sum()
        dict_airline[i]['rpk_internacional'] = rpk_internacional
        
        #ASK Internacional
        ask_internacional = internacional['ASK']
        ask_internacional = ask_internacional.astype(float)
        ask_internacional = ask_internacional.sum()
        dict_airline[i]['ask_internacional'] = ask_internacional
        
        #Decolagens
        decolagens = df_airline['DECOLAGENS']
        decolagens  = decolagens.astype(int)
        decolagens = decolagens.sum()
        dict_airline[i]['decolagens'] = decolagens
        
        #Etapa Media de Voo (KM)
        etapa_media_voo = df_airline['DISTÂNCIA VOADA (KM)'].sum()/df_airline['DECOLAGENS'].sum()
        dict_airline[i]['etapa_media_voo'] =  etapa_media_voo
        etapa_media_voo
        
        #Combustível de aviação
        combustivel = df_airline['COMBUSTÍVEL (LITROS)']
        combustivel  = combustivel.astype(int)
        combustivel= combustivel.sum()
        dict_airline[i]['combustivel'] = combustivel
        

    df = pd.DataFrame(dict_airline).T

    return df

def plot_graf(df, name):
    df_print = df[['assentos_totais', 'passageiros_pgs', 'assentos_domestica',
               'passageiros_pgs_domestica',
               'assentos_internacional', 'passageiros_pgs_internacional']]
    df_print.reset_index(inplace=True)

    fig = px.bar(df_print, x='index', y=['assentos_totais', 'passageiros_pgs','assentos_domestica','passageiros_pgs_domestica','assentos_internacional', 'passageiros_pgs_internacional'], 
                 barmode='group', title='Estatísticas de transporte aéreo no Brasil')
    fig.update_layout(
    xaxis_title='Ano',
    yaxis_title='Passageiros',
    title={
    'text': f'Estatísticas de transporte aéreo da {name}',
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
    fig.show()
    
def ind_trafego(df, ano, name):
    
    dict_ind_op_fin = {
        'RPK - Total': round(df.loc[ano]['rpk_total'] / 1_000_000, 3),
        'RPK - Mercado Doméstico': round(df.loc[ano]['rpk_domestica'] / 1_000_000, 3),
        'RPK - Mercado Internacional': round(df.loc[ano]['rpk_internacional'] / 1_000_000, 3),
        'ASK - Total': round(df.loc[ano]['ask_total'] / 1_000_000, 3),
        'ASK - Mercado Doméstico': round(df.loc[ano]['ask_domestica'] / 1_000_000, 3),
        'ASK - Mercado Internacional': round(df.loc[ano]['ask_internacional'] / 1_000_000, 3),
        'Taxa de Ocupação - Total (%)': round(df.loc[ano]['passageiros_pgs'] / df.loc[ano]['assentos_totais'] * 100, 1),
        'Taxa de Ocupação - Mercado Doméstico (%)': round(df.loc[ano]['passageiros_pgs_domestica'] / df.loc[ano]['assentos_domestica'] * 100, 1),
        'Taxa de Ocupação - Mercado Internacional (%)': round(df.loc[ano]['passageiros_pgs_internacional'] / df.loc[ano]['assentos_internacional'] * 100, 1)
    }
    if ano > 2010:
        
        dict_ind_op_fin_ant = {
        'RPK - Total': round(df.loc[ano-1]['rpk_total'] / 1_000_000, 3),
        'RPK - Mercado Doméstico': round(df.loc[ano-1]['rpk_domestica'] / 1_000_000, 3),
        'RPK - Mercado Internacional': round(df.loc[ano-1]['rpk_internacional'] / 1_000_000, 3),
        'ASK - Total': round(df.loc[ano-1]['ask_total'] / 1_000_000, 3),
        'ASK - Mercado Doméstico': round(df.loc[ano-1]['ask_domestica'] / 1_000_000, 3),
        'ASK - Mercado Internacional': round(df.loc[ano-1]['ask_internacional'] / 1_000_000, 3),
        'Taxa de Ocupação - Total (%)': round(df.loc[ano-1]['passageiros_pgs'] / df.loc[ano-1]['assentos_totais'] * 100, 1),
        'Taxa de Ocupação - Mercado Doméstico (%)': round(df.loc[ano-1]['passageiros_pgs_domestica'] / df.loc[ano-1]['assentos_domestica'] * 100, 1),
        'Taxa de Ocupação - Mercado Internacional (%)': round(df.loc[ano-1]['passageiros_pgs_internacional'] / df.loc[ano-1]['assentos_internacional'] * 100, 1)
        }
        
        df = pd.DataFrame({ano: list(dict_ind_op_fin.values()), ano-1: list(dict_ind_op_fin_ant.values())}, index=[
            'RPK - Total','RPK - Mercado Doméstico','RPK - Mercado Internacional','ASK - Total','ASK - Mercado Doméstico','ASK - Mercado Internacional',
            'Taxa de Ocupação - Total (%)','Taxa de Ocupação - Mercado Doméstico (%)','Taxa de Ocupação - Mercado Internacional (%)'
        ])
        
        df[f'% Var'] = ((df[ano] / df[ano-1]) - 1) * 100
        
        df = df.rename_axis(f'Dados de Tráfego - {name} (em Milhões)')
        
         
        
        print('''Assentos-quilômetro oferecidos (ASK) :
        Número de assentos disponíveis multiplicado pelos quilômetros voados.''')
        print('''\n Passageiros-quilômetro transportados (RPK) :
        Passageiros pagantes transportados em um quilômetro. O RPK é calculado ao multiplicar-se o número de 
        passageiros pagantes pelos quilômetros voados.''')
        return df
    else:
        df = pd.DataFrame.from_dict(dict_ind_op_fin, orient='index', columns=[ano])
        df[f'Dados de Tráfego - {name} (em Milhões)'] = list(dict_ind_op_fin.keys())
        df = df.set_index([f'Dados de Tráfego - {name} (em Milhões)'])
        
        return df

def ind_operacional(df, ano, name):
    
    dict_ind_operacional = {
        "Passageiros Pagantes ('000)": round(df.loc[ano]['passageiros_pgs'] / 1_000_000, 3),
        "Passageiros Pagantes Doméstica ('000)": round(df.loc[ano]['passageiros_pgs_domestica'] / 1_000_000, 3),
        "Passageiros Pagantes Internacional ('000)": round(df.loc[ano]['passageiros_pgs_internacional'] / 1_000_000, 3),
        "Total Assentos Disponibilizados ('000)": round(df.loc[ano]['assentos_totais'] / 1_000_000, 3),
        "Total Assentos Disponibilizados Doméstica ('000)": round(df.loc[ano]['assentos_domestica'] / 1_000_000, 3),
        "Total Assentos Disponibilizados Internacional ('000)": round(df.loc[ano]['assentos_internacional'] / 1_000_000, 3),
        "Decolagens": round(df.loc[ano]['decolagens'] / 1_000, 3),
        "Etapa Média de Voo (km)": round(df.loc[ano]['etapa_media_voo'] / 1_000, 3),
        "Combustível de aviação (milhares litros)": round(df.loc[ano]['combustivel'] / 1_000, 0)
    }
    if ano > 2010:
        
        dict_ind_operacional_ant = {
        "Passageiros Pagantes ('000)": round(df.loc[ano-1]['passageiros_pgs'] / 1_000_000, 3),
        "Passageiros Pagantes Doméstica ('000)": round(df.loc[ano-1]['passageiros_pgs_domestica'] / 1_000_000, 3),
        "Passageiros Pagantes Internacional ('000)": round(df.loc[ano-1]['passageiros_pgs_internacional'] / 1_000_000, 3),
        "Total Assentos Disponibilizados ('000)": round(df.loc[ano-1]['assentos_totais'] / 1_000_000, 3),
        "Total Assentos Disponibilizados Doméstica ('000)": round(df.loc[ano-1]['assentos_domestica'] / 1_000_000, 3),
        "Total Assentos Disponibilizados Internacional ('000)": round(df.loc[ano-1]['assentos_internacional'] / 1_000_000, 3),
        "Decolagens": round(df.loc[ano-1]['decolagens'] / 1_000, 3),
        "Etapa Média de Voo (km)": round(df.loc[ano-1]['etapa_media_voo'] / 1_000, 3),
        "Combustível de aviação (milhares litros)": round(df.loc[ano-1]['combustivel'] / 1_000, 0)
        }
        
        df = pd.DataFrame({ano: list(dict_ind_operacional.values()), ano-1: list(dict_ind_operacional_ant.values())}, index=[
          "Passageiros Pagantes ('000)",  "Passageiros Pagantes Doméstica ('000)","Passageiros Pagantes Internacional ('000)",
          "Total Assentos Disponibilizados ('000)",  "Total Assentos Disponibilizados Doméstica ('000)", "Total Assentos Disponibilizados Internacional ('000)",
          "Decolagens", "Etapa Média de Voo (km)", "Combustível de aviação (milhares litros)"
        ])
        
        df[f'% Var'] = ((df[ano] / df[ano-1]) - 1) * 100
        
        df = df.rename_axis(f'Dados Operacionais')
        
        return df
    else:
        df = pd.DataFrame.from_dict( dict_ind_operacional, orient='index', columns=[ano])
        df[f'Dados Operacionais - {name}'] = list(dict_ind_op_fin.keys())
        df = df.set_index([f'Dados Operacionais - {name}'])
        
        return df
    

In [214]:
df_GOL = extract_airline_data('GLO', 'GOL')
plot_graf(df_GOL, 'GOL')
display(ind_trafego(df_GOL,2022,'GOL'))
ind_operacional(df_GOL,2022,'GOL')

Assentos-quilômetro oferecidos (ASK) :
        Número de assentos disponíveis multiplicado pelos quilômetros voados.

 Passageiros-quilômetro transportados (RPK) :
        Passageiros pagantes transportados em um quilômetro. O RPK é calculado ao multiplicar-se o número de 
        passageiros pagantes pelos quilômetros voados.


,2022,2021,% Var
Dados de Tráfego - GOL (em Milhões),,,
RPK - Total,32627.345,22237.493,46.722227
RPK - Mercado Doméstico,30154.802,22115.232,36.353089
RPK - Mercado Internacional,2472.542,122.261,1922.347273
ASK - Total,40788.556,27132.414,50.331467
ASK - Mercado Doméstico,37714.370,26962.930,39.874895
ASK - Mercado Internacional,3074.186,169.484,1713.850275
Taxa de Ocupação - Total (%),77.200,79.900,-3.379224
Taxa de Ocupação - Mercado Doméstico (%),77.100,79.900,-3.504380
Taxa de Ocupação - Mercado Internacional (%),79.100,68.100,16.152717


,2022,2021,% Var
Dados Operacionais,,,
Passageiros Pagantes ('000),27.213,18.813,44.649976
Passageiros Pagantes Doméstica ('000),26.300,18.772,40.102280
Passageiros Pagantes Internacional ('000),0.913,0.041,2126.829268
Total Assentos Disponibilizados ('000),35.254,23.559,49.641326
Total Assentos Disponibilizados Doméstica ('000),34.100,23.500,45.106383
Total Assentos Disponibilizados Internacional ('000),1.154,0.059,1855.932203
Decolagens,203.647,134.636,51.257465
Etapa Média de Voo (km),1.143,1.137,0.527704
Combustível de aviação (milhares litros),1031107.000,696014.000,48.144578


In [215]:
df_AZUL = extract_airline_data('AZU', 'AZUL')
plot_graf(df_AZUL, 'AZUL')
display(ind_trafego(df_AZUL,2022,'AZUL'))
ind_operacional(df_AZUL,2022,'AZUL')


Assentos-quilômetro oferecidos (ASK) :
        Número de assentos disponíveis multiplicado pelos quilômetros voados.

 Passageiros-quilômetro transportados (RPK) :
        Passageiros pagantes transportados em um quilômetro. O RPK é calculado ao multiplicar-se o número de 
        passageiros pagantes pelos quilômetros voados.


,2022,2021,% Var
Dados de Tráfego - AZUL (em Milhões),,,
RPK - Total,31234.939,24684.900,26.534598
RPK - Mercado Doméstico,26203.251,23300.829,12.456304
RPK - Mercado Internacional,5031.688,1384.070,263.542884
ASK - Total,39185.033,31128.664,25.880870
ASK - Mercado Doméstico,33228.850,29186.021,13.851936
ASK - Mercado Internacional,5956.183,1942.643,206.602037
Taxa de Ocupação - Total (%),76.200,77.700,-1.930502
Taxa de Ocupação - Mercado Doméstico (%),76.000,77.800,-2.313625
Taxa de Ocupação - Mercado Internacional (%),83.200,69.500,19.712230


,2022,2021,% Var
Dados Operacionais,,,
Passageiros Pagantes ('000),27.176,22.981,18.254210
Passageiros Pagantes Doméstica ('000),26.376,22.776,15.806112
Passageiros Pagantes Internacional ('000),0.799,0.205,289.756098
Total Assentos Disponibilizados ('000),35.671,29.575,20.612003
Total Assentos Disponibilizados Doméstica ('000),34.709,29.280,18.541667
Total Assentos Disponibilizados Internacional ('000),0.961,0.295,225.762712
Decolagens,283.783,232.067,22.284944
Etapa Média de Voo (km),0.915,0.904,1.216814
Combustível de aviação (milhares litros),1146747.000,954166.000,20.183176


In [217]:
df_TAM = extract_airline_data('TAM', 'TAM')
plot_graf(df_TAM, 'TAM')
display(ind_trafego(df_TAM,2022,'TAM'))
ind_operacional(df_TAM,2022,'TAM')

Assentos-quilômetro oferecidos (ASK) :
        Número de assentos disponíveis multiplicado pelos quilômetros voados.

 Passageiros-quilômetro transportados (RPK) :
        Passageiros pagantes transportados em um quilômetro. O RPK é calculado ao multiplicar-se o número de 
        passageiros pagantes pelos quilômetros voados.


,2022,2021,% Var
Dados de Tráfego - TAM (em Milhões),,,
RPK - Total,48948.742,27763.543,76.305819
RPK - Mercado Doméstico,32646.001,23544.108,38.658899
RPK - Mercado Internacional,16302.741,4219.436,286.372515
ASK - Total,60289.258,36054.608,67.216512
ASK - Mercado Doméstico,40916.262,29326.867,39.518013
ASK - Mercado Internacional,19372.996,6727.741,187.956923
Taxa de Ocupação - Total (%),77.300,78.700,-1.778907
Taxa de Ocupação - Mercado Doméstico (%),77.000,79.400,-3.022670
Taxa de Ocupação - Mercado Internacional (%),81.600,62.900,29.729730


,2022,2021,% Var
Dados Operacionais,,,
Passageiros Pagantes ('000),31.144,20.572,51.390239
Passageiros Pagantes Doméstica ('000),28.664,19.907,43.989551
Passageiros Pagantes Internacional ('000),2.481,0.664,273.644578
Total Assentos Disponibilizados ('000),40.287,26.141,54.114227
Total Assentos Disponibilizados Doméstica ('000),37.246,25.085,48.479171
Total Assentos Disponibilizados Internacional ('000),3.041,1.055,188.246445
Decolagens,222.814,148.584,49.958273
Etapa Média de Voo (km),1.287,1.276,0.862069
Combustível de aviação (milhares litros),1764837.000,1092478.000,61.544397
